In [3]:
def registrar_usuario(
    nombre,
    apellido,
    documento,
    placa,
    usuarios_registrados
):
    errores = False

    # Nombre
    if len(nombre) < 3:
        print('Error, el nombre debe tener minimo 3 letras')
        errores = True
    if not nombre.isalpha():
        print('Error, solo debe contener letras')
        errores = True

    # Apellido
    if len(apellido) < 3:
        print('Error, el apellido debe tener minimo 3 letras')
        errores = True
    if not apellido.isalpha():
        print('Error, solo debe contener letras')
        errores = True

    # Documento
    documento = str(documento)
    if not documento.isdigit():
        print('Error, el documento debe contener solo numeros')
        errores = True
    if not 3 <= len(documento) <= 15:
        print('Error, el documento debe tener minimo 3 numeros y maximo 15')
        errores = True

    # Placa
    if not len(placa) == 6:
        print('Error, la placa debe contener exactamente 6 caracteres')
        errores = True
    if not (placa[:3].isalpha() and placa[3:].isdigit()):
        print('La placa debe contener 3 letras y 3 numeros respectivamente')
        errores = True

    if errores:
        return None

    # Registro de usuario
    else:
        usuario = {
            "nombre": nombre.capitalize(),
            "apellido": apellido.capitalize(),
            "documento": documento,
            "placa": placa.upper()
        }
        usuarios_registrados.append(usuario)

    with open(ruta, "a") as archivo:
        for dic in usuarios_registrados:
            archivo.write(str(dic) + "\n")

    return "Usuario registrado exitosamente!", usuario

In [4]:
def ingresar_vehiculo(
        documento,
        usuarios_registrados
):

    # Verificar usuario
    usuario_encontrado = None
    for u in usuarios_registrados:
        if u["documento"] == documento:
            usuario_encontrado = u
            print('Usuario existente.')
            break
    if not usuario_encontrado:
        return "Error: Usuario no registrado."

    # Asignar puesto
    if usuario_encontrado:
        if espacios_disponibles:
            espacio_asignado = espacios_disponibles.pop(0)
            usuario_encontrado["espacio"] = espacio_asignado
            hora_entrada = datetime.now().strftime("%I:%M:%S %p")
            usuario_encontrado["hora de entrada"] = hora_entrada

            return usuario_encontrado
        else:
            return "No hay espacios disponibles."

In [5]:
def retirar_vehiculo(
    documento,
    placa,
    usuarios_registrados,
    espacios_disponibles,
    tiempos_estancia
):

    # Verificar documento
    usuario_encontrado = None
    for u in usuarios_registrados:
        if u["documento"] == documento:
            usuario_encontrado = u
            break
    if not usuario_encontrado:
        return "Error: Usuario no registrado."

    if usuario_encontrado["placa"] != placa:
        return "Error: La placa no coincide con el documento."

    if "espacio" not in usuario_encontrado:
        return "Error: El vehículo no tiene un espacio asignado actualmente."

    # Liberar espacio
    else:
        espacio_liberado = usuario_encontrado["espacio"]
        espacios_disponibles.append(espacio_liberado)
        del usuario_encontrado["espacio"]

        # Asigar hora de entrada
        formato = "%I:%M:%S %p"
        entrada = datetime.strptime(
            usuario_encontrado["hora de entrada"], formato
        ).time()
        fecha_actual = datetime.now().date()
        entrada = datetime.combine(fecha_actual, entrada)
        salida = datetime.now()
        tiempo_transcurrido = salida - entrada
        tiempos_estancia.append({
            'tiempo_transcurrido': tiempo_transcurrido,
            'Vehículo': usuario_encontrado["placa"]
        })
        del usuario_encontrado['hora de entrada']

        # Calcular precio
        valor_hora = 7000
        valor_cuartos = 500
        total_segundos = tiempo_transcurrido.total_seconds()
        horas = int(total_segundos // 3600)
        segundos_restantes = total_segundos - (horas * 3600)
        cuartos = int(segundos_restantes // 900)
        total = (horas * valor_hora) + (cuartos * valor_cuartos)

        if total < valor_hora:
            total = valor_hora

        return f"usuario: {usuario_encontrado}, pago: {total}", total

In [6]:
def menu_admin(
    usuarios_registrados,
    vehiculos_registrados,
    vehiculos_retirados,
):
    vehiculos_sin_retirar = 0
    total_espacios = 50

    for usuario in usuarios_registrados:
        if "espacio" in usuario:
            vehiculos_sin_retirar += 1

    celdas_libres = total_espacios - vehiculos_sin_retirar

    # Calcular promedio de tiempo
    if not tiempos_estancia:
        print("\nNo hay vehículos retirados aún.")
        tiempo_promedio = "No hay datos"
        usuario_max = {"nombre": "N/A", "apellido": "", "placa": ""}
        usuario_min = {"nombre": "N/A", "apellido": "", "placa": ""}
    else:
        suma = sum([
            t['tiempo_transcurrido'].total_seconds()
            for t in tiempos_estancia
        ])
        promedio_segundos = suma / len(tiempos_estancia)
        tiempo_promedio = (
            f"{int(promedio_segundos // 3600)}h "
            f"{int((promedio_segundos % 3600) // 60)}min"
        )
        menor = min(tiempos_estancia, key=lambda t: t['tiempo_transcurrido'])
        mayor = max(tiempos_estancia, key=lambda t: t['tiempo_transcurrido'])
        usuario_min = menor['Vehiculo']
        usuario_max = mayor['Vehiculo']

    print("\n RESUMEN DEL PARQUEADERO \n")
    print(f"Total de vehículos registrados: {vehiculos_registrados}")
    print(f"Total de vehículos retirados: {vehiculos_retirados}")
    print(f"Total de vehículos sin retirar: {vehiculos_sin_retirar}")
    print(f"Total pago de vehículos retirados: ${ganancias}")
    print(f"Tiempo promedio de estancia por vehículo: {tiempo_promedio}\n")

    print("Lista de usuarios registrados:")
    for usuario in usuarios_registrados:
        print(
            f"{usuario['nombre']} {usuario['apellido']}"
            f"Placa: {usuario['placa']}, Doc: {usuario['documento']}"
        )

    print("\nVehículo con mayor tiempo de parqueo:")
    print(usuario_max)

    print("Vehículo con menor tiempo de parqueo:")
    print(usuario_min)

    print(f"\nCeldas libres actualmente: {celdas_libres}")

In [7]:
from datetime import datetime
import ast
correo_admin = 'fabian@udea.edu.co'
contraseña_admin = 'meravuelta'
usuarios_registrados = []
tiempos_estancia = []
espacios_disponibles = list(range(1, 51))
ruta = 'C:/Users/centr/OneDrive/Documentos/Programacion/usuarios.txt'
vehiculos_registrados = 0
vehiculos_retirados = 0
ganancias = 0

# Usuarios Guardados
with open(ruta, "r") as archivo:
    for linea in archivo:
        if linea.strip():
            # convierte texto a diccionario
            usuario = ast.literal_eval(linea.strip())
            usuarios_registrados.append(usuario)

In [10]:
respuesta = ''

print("\n" + "="*70)
print("🚗🚗🚗          B I E N V E N I D O   A")
print(r"""
   _____ _    _ ______  _____ _  __     _____   ___ _______ _   _ _______
  / ____| |  | |  ____|/ ____| |/ /    |  __ \ / _ \|_   _ | \ | |__   __|
 | |    | |__| | |__  | |    | ' /_____| |__) | | | | | |  |  \| |  | |   
 | |    |  __  |  __| | |    |  <______|  ___/| | | | | |  | . ` |  | |   
 | |____| |  | | |____| |____| . \     | |    | |_| |_| |_ | |\  |  | |   
  \_____|_|  |_|______|\_____|_|\_\    |_|     \___/|_____||_| \_|  |_|   
    """)
print("🚦               ¡Bienvenido a CHECK POINT!               🚦")
print("="*70 + "\n")

while respuesta != 5:

    respuesta = int(input(
        'Ingrese el número de la función que desea realizar:\n'
        '1 para registrarse\n'
        '2 para ingresar vehículo\n'
        '3 para retirar vehículo\n'
        '4 para ingresar al menú de administrador\n'
        '5 para salir\n '
    ))

    if respuesta not in [1, 2, 3, 4, 5]:
        print("Opción no válida. Por favor, ingrese un número entre 1 y 4.")
        continue

    if respuesta == 1:
        nombre = input('Ingrese su nombre: ')
        apellido = input('Ingrese su apellido: ')
        documento = input('Ingrese su documento: ')
        placa = input('Ingrese la placa del vehículo: ')
        mensaje = registrar_usuario(
            nombre, apellido, documento, placa, usuarios_registrados
        )
        print(mensaje)

    elif respuesta == 2:
        documento = input('Ingrese su documento: ')
        mensaje = ingresar_vehiculo(documento, usuarios_registrados)
        if 'Error' not in mensaje:
            vehiculos_registrados += 1
        print(mensaje)

    elif respuesta == 3:
        documento = input('Ingrese su documento: ')
        placa = input('Ingrese la placa del vehículo: ').upper()
        mensaje, total = retirar_vehiculo(
            documento, placa, usuarios_registrados,
            espacios_disponibles, tiempos_estancia
        )
        if 'Error' not in mensaje:
            vehiculos_retirados += 1
            ganancias += total
        print(mensaje)

    elif respuesta == 4:
        correo = str(input('Ingrese el correo de administrador: '))
        if correo == correo_admin:
            contraseña = str(input('Ingrese la contraseña: '))
            if contraseña == contraseña_admin:
                mensaje = menu_admin(
                    usuarios_registrados, vehiculos_registrados,
                    vehiculos_retirados, ganancias
                )
                print(mensaje)
            else:
                print('Contraseña incorrecta')
        else:
            print('Correo incorrecto')

print("Gracias por usar el sistema de parqueadero! Hasta pronto.")


🚗🚗🚗          B I E N V E N I D O   A

   _____ _    _ ______  _____ _  __     _____   ___ _______ _   _ _______
  / ____| |  | |  ____|/ ____| |/ /    |  __ \ / _ \|_   _ | \ | |__   __|
 | |    | |__| | |__  | |    | ' /_____| |__) | | | | | |  |  \| |  | |   
 | |    |  __  |  __| | |    |  <______|  ___/| | | | | |  | . ` |  | |   
 | |____| |  | | |____| |____| . \     | |    | |_| |_| |_ | |\  |  | |   
  \_____|_|  |_|______|\_____|_|\_\    |_|     \___/|_____||_| \_|  |_|   
    
🚦               ¡Bienvenido a CHECK POINT!               🚦

Usuario existente.
{'nombre': 'Fabian', 'apellido': 'Zambrano', 'documento': '1091970950', 'placa': 'EHP000', 'espacio': 2, 'hora de entrada': '10:21:05 PM'}
Gracias por usar el sistema de parqueadero! Hasta pronto.
